In [19]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2
import urllib.parse
import datetime as dt
from urllib.parse import quote_plus as urlquote

In [20]:
covid_vaccine_csv = "../data/covid_vaccine.csv"
covid_cases_csv = "../data/covid_cases.csv"

vaccine_df = pd.read_csv(covid_vaccine_csv)
cases_df = pd.read_csv(covid_cases_csv)

##covid_df = pd.merge(vaccine_df, cases_df)

In [21]:
vaccine_df

,date,location,total_vaccinations,total_distributed,people_vaccinated,people_fully_vaccinated_per_hundred,total_vaccinations_per_hundred,people_fully_vaccinated,people_vaccinated_per_hundred,distributed_per_hundred,daily_vaccinations_raw,daily_vaccinations,daily_vaccinations_per_million,share_doses_used
0,2021-01-12,Alabama,78134.0,377025.0,70861.0,0.15,1.59,7270.0,1.44,7.69,NaN,NaN,NaN,0.207
1,2021-01-13,Alabama,84040.0,378975.0,74792.0,0.19,1.71,9245.0,1.52,7.73,5906.0,5906.0,1205.0,0.222
2,2021-01-14,Alabama,92300.0,435350.0,80480.0,NaN,1.88,NaN,1.64,8.88,8260.0,7083.0,1445.0,0.212
3,2021-01-15,Alabama,100567.0,444650.0,86956.0,0.27,2.05,13488.0,1.77,9.07,8267.0,7478.0,1525.0,0.226
4,2021-01-16,Alabama,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7557.0,7498.0,1529.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3333,2021-02-27,Wyoming,151584.0,195855.0,98484.0,9.09,26.19,52608.0,17.02,33.84,6007.0,3218.0,5560.0,0.774
3334,2021-02-28,Wyoming,156397.0,195855.0,100902.0,9.50,27.02,54962.0,17.43,33.84,4813.0,3847.0,6647.0,0.799
3335,2021-03-01,Wyoming,156741.0,195855.0,100903.0,9.56,27.08,55305.0,17.43,33.84,344.0,3015.0,5209.0,0.800
3336,2021-03-02,Wyoming,156776.0,211415.0,100911.0,9.56,27.09,55332.0,17.44,36.53,35.0,2945.0,5088.0,0.742


In [22]:
vaccine_df.drop(columns=["total_distributed", "daily_vaccinations_raw", "share_doses_used"])


,date,location,total_vaccinations,people_vaccinated,people_fully_vaccinated_per_hundred,total_vaccinations_per_hundred,people_fully_vaccinated,people_vaccinated_per_hundred,distributed_per_hundred,daily_vaccinations,daily_vaccinations_per_million
0,2021-01-12,Alabama,78134.0,70861.0,0.15,1.59,7270.0,1.44,7.69,NaN,NaN
1,2021-01-13,Alabama,84040.0,74792.0,0.19,1.71,9245.0,1.52,7.73,5906.0,1205.0
2,2021-01-14,Alabama,92300.0,80480.0,NaN,1.88,NaN,1.64,8.88,7083.0,1445.0
3,2021-01-15,Alabama,100567.0,86956.0,0.27,2.05,13488.0,1.77,9.07,7478.0,1525.0
4,2021-01-16,Alabama,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7498.0,1529.0
...,...,...,...,...,...,...,...,...,...,...,...
3333,2021-02-27,Wyoming,151584.0,98484.0,9.09,26.19,52608.0,17.02,33.84,3218.0,5560.0
3334,2021-02-28,Wyoming,156397.0,100902.0,9.50,27.02,54962.0,17.43,33.84,3847.0,6647.0
3335,2021-03-01,Wyoming,156741.0,100903.0,9.56,27.08,55305.0,17.43,33.84,3015.0,5209.0
3336,2021-03-02,Wyoming,156776.0,100911.0,9.56,27.09,55332.0,17.44,36.53,2945.0,5088.0


In [23]:
cases_df


,date,state,death,deathConfirmed,deathIncrease,deathProbable,hospitalized,hospitalizedCumulative,hospitalizedCurrently,hospitalizedIncrease,...,totalTestResults,totalTestResultsIncrease,totalTestsAntibody,totalTestsAntigen,totalTestsPeopleAntibody,totalTestsPeopleAntigen,totalTestsPeopleViral,totalTestsPeopleViralIncrease,totalTestsViral,totalTestsViralIncrease
0,2021-03-05,AK,305.0,NaN,2,NaN,1293.0,1293.0,33.0,3,...,1731628.0,7144,NaN,NaN,NaN,NaN,NaN,0,1731628.0,7144
1,2021-03-05,AL,10122.0,7943.0,28,2179.0,45976.0,45976.0,526.0,169,...,2316127.0,4920,NaN,NaN,119005.0,NaN,2316127.0,4920,NaN,0
2,2021-03-05,AR,5283.0,4291.0,10,992.0,14903.0,14903.0,359.0,27,...,2728103.0,8874,NaN,NaN,NaN,479119.0,NaN,0,2728103.0,8874
3,2021-03-05,AS,0.0,NaN,0,NaN,NaN,NaN,NaN,0,...,2140.0,0,NaN,NaN,NaN,NaN,NaN,0,2140.0,0
4,2021-03-05,AZ,16269.0,14339.0,84,1930.0,57819.0,57819.0,1043.0,72,...,7827570.0,51727,441664.0,NaN,NaN,NaN,3818450.0,20834,7827570.0,51727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20663,2020-01-17,WA,NaN,NaN,0,NaN,NaN,NaN,NaN,0,...,NaN,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0
20664,2020-01-16,WA,NaN,NaN,0,NaN,NaN,NaN,NaN,0,...,NaN,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0
20665,2020-01-15,WA,NaN,NaN,0,NaN,NaN,NaN,NaN,0,...,NaN,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0
20666,2020-01-14,WA,NaN,NaN,0,NaN,NaN,NaN,NaN,0,...,NaN,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0


In [25]:
start_date = "2021-1-12"
end_date = "2021-3-5"

after_start_date = cases_df["date"] >= start_date
before_end_date = cases_df["date"] <= end_date
between_two_dates = after_start_date & before_end_date
filtered_dates = cases_df.loc[between_two_dates]
cases_df

,date,state,death,deathConfirmed,deathIncrease,deathProbable,hospitalized,hospitalizedCumulative,hospitalizedCurrently,hospitalizedIncrease,...,totalTestResults,totalTestResultsIncrease,totalTestsAntibody,totalTestsAntigen,totalTestsPeopleAntibody,totalTestsPeopleAntigen,totalTestsPeopleViral,totalTestsPeopleViralIncrease,totalTestsViral,totalTestsViralIncrease
0,2021-03-05,AK,305.0,NaN,2,NaN,1293.0,1293.0,33.0,3,...,1731628.0,7144,NaN,NaN,NaN,NaN,NaN,0,1731628.0,7144
1,2021-03-05,AL,10122.0,7943.0,28,2179.0,45976.0,45976.0,526.0,169,...,2316127.0,4920,NaN,NaN,119005.0,NaN,2316127.0,4920,NaN,0
2,2021-03-05,AR,5283.0,4291.0,10,992.0,14903.0,14903.0,359.0,27,...,2728103.0,8874,NaN,NaN,NaN,479119.0,NaN,0,2728103.0,8874
3,2021-03-05,AS,0.0,NaN,0,NaN,NaN,NaN,NaN,0,...,2140.0,0,NaN,NaN,NaN,NaN,NaN,0,2140.0,0
4,2021-03-05,AZ,16269.0,14339.0,84,1930.0,57819.0,57819.0,1043.0,72,...,7827570.0,51727,441664.0,NaN,NaN,NaN,3818450.0,20834,7827570.0,51727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20663,2020-01-17,WA,NaN,NaN,0,NaN,NaN,NaN,NaN,0,...,NaN,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0
20664,2020-01-16,WA,NaN,NaN,0,NaN,NaN,NaN,NaN,0,...,NaN,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0
20665,2020-01-15,WA,NaN,NaN,0,NaN,NaN,NaN,NaN,0,...,NaN,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0
20666,2020-01-14,WA,NaN,NaN,0,NaN,NaN,NaN,NaN,0,...,NaN,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0


In [7]:
cases_df.drop(columns=["deathConfirmed","deathProbable","","hospitalizedCumulative", "hospitalizedCurrently", "pnew_death", "created_at", "consent_cases", "consent_deaths"])


,submission_date,state,tot_cases,conf_cases,new_case,tot_death,conf_death,new_death
0,05/04/2020,KY,5245,5245.0,115,261,261.0,8
1,03/23/2020,NH,101,NaN,23,1,NaN,1
2,10/28/2020,RI,31944,NaN,499,1192,NaN,4
3,05/29/2020,NJ,159196,NaN,1673,11531,11531.0,130
4,02/14/2020,DC,0,NaN,0,0,NaN,0
...,...,...,...,...,...,...,...,...
24535,03/23/2020,RI,106,NaN,9,0,NaN,0
24536,10/09/2020,GU,2989,NaN,0,58,NaN,0
24537,07/12/2020,DC,10847,NaN,46,568,NaN,0
24538,03/05/2020,MI,0,NaN,0,0,NaN,0


In [15]:
connection_string = "postgres:Cellxan69$$@localhost:5432/covid_db"
engine = create_engine(f'postgresql://{connection_string}')

In [16]:
engine.table_names()


['vaccine', 'cases']

In [17]:
vaccine_df.to_sql(name= "vaccine", con=engine, if_exists= 'append', index= True)

In [18]:
cases_df.to_sql(name= "cases", con=engine, if_exists= 'append', index= True)